.env

In [3]:
%env OW_API_KEY=4a576347b547a6d8f5e8796a7519f7e8
%env NEWS_API_KEY=d48456bca5eb49c8a822fa59e17b259c

env: OW_API_KEY=4a576347b547a6d8f5e8796a7519f7e8
env: NEWS_API_KEY=d48456bca5eb49c8a822fa59e17b259c


Install and Import

In [4]:
!pip install requests python-dotenv


In [5]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()


False

Planner (next launch from SpaceX)

In [6]:
def get_next_launch():
    """Query SpaceX API to find the next launch's site and datetime."""
    res = requests.get("https://api.spacexdata.com/v4/launches/next")
    launch = res.json()
    return {
        "location_id": launch['launchpad'],
        "time": launch['date_utc']
    }


Weather (using launch's location)

In [11]:
def get_weather(launch):
    """Using launch's location, find the forecast for launch time."""
    API_KEY = os.getenv("OW_API_KEY")
    launchpad_url = f"https://api.spacexdata.com/v4/launchpads/{launch['location_id']}"
    res = requests.get(launchpad_url)
    launchpad = res.json()
    lat, lon = launchpad['latitude'], launchpad['longitude']

    res = requests.get(
        "https://api.openweathermap.org/data/2.5/forecast",
        params={"appid": API_KEY, "lat": lat, "lon": lon, "units": "metric"}
    )
    forecast = res.json()

    # Add error handling to check if the API call was successful and the expected keys exist
    if res.status_code != 200 or 'list' not in forecast:
        print(f"Error fetching weather data: {forecast}")
        # You might want to raise an exception or return a default value here
        # For now, returning an empty structure to prevent the KeyError in summarize
        return {
            "latitude": lat,
            "longitude": lon,
            "forecast": {"list": [{"weather": [{"description": "N/A"}], "main": {"temp": "N/A"}}]}
        }

    return {
        "latitude": lat,
        "longitude": lon,
        "forecast": forecast
    }

News (using launch's details)

In [12]:
# The rest of the code remains the same
def get_next_launch():
    """Query SpaceX API to find the next launch's site and datetime."""
    res = requests.get("https://api.spacexdata.com/v4/launches/next")
    launch = res.json()
    return {
        "location_id": launch['launchpad'],
        "time": launch['date_utc']
    }

def get_news(launch):
    """Search News API for articles related to SpaceX launch."""
    API_KEY = os.getenv("NEWS_API_KEY")
    res = requests.get("https://newsapi.org/v2/everything",
                        params={"q": "SpaceX launch", "apiKey": API_KEY})
    articles = res.json()["articles"][:5]  # first 5 articles
    return articles

Combine into Summary

In [13]:
def summarize(launch, weather, news):
    """Combine all into a summary."""
    # Use .get() with default values to safely access nested keys
    weather_description = weather.get('forecast', {}).get('list', [{}])[0].get('weather', [{}])[0].get('description', 'N/A')
    temp = weather.get('forecast', {}).get('list', [{}])[0].get('main', {}).get('temp', 'N/A')

    summary = f"""🚀 Next launch:
    - Location (latitude:{weather.get('latitude', 'N/A')} , longitude:{weather.get('longitude', 'N/A')})
    - Time: {launch.get('time', 'N/A')}
    - Weather forecast at launch: {weather_description} with temperature {temp}°C

    📰 News related to launch:
    """
    for i, article in enumerate(news, 1):
        # Use .get() with default values for article details as well
        title = article.get('title', 'No Title')
        url = article.get('url', '#')
        summary += f"{i}. {title} - {url}\n"

    return summary

Run pipeline

In [14]:
launch = get_next_launch()
weather = get_weather(launch)
news = get_news(launch)

final_report = summarize(launch, weather, news)
print(final_report)

Error fetching weather data: {'cod': 401, 'message': 'Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.'}
🚀 Next launch:
    - Location (latitude:28.6080585 , longitude:-80.6039558)
    - Time: 2022-11-01T13:41:00.000Z
    - Weather forecast at launch: N/A with temperature N/A°C

    📰 News related to launch:
    1. SpaceX Tests Starship Fixes After Back-to-Back Failures - https://www.wired.com/story/spacex-tests-starship-fixes-after-back-to-back-failures/
2. Florida Environmentalists Sound Alarm on SpaceX Bid to Launch 120 Rockets a Year - https://gizmodo.com/florida-environmentalists-sound-alarm-on-spacex-bid-to-launch-120-rockets-a-year-2000603090
3. Watch Live as SpaceX Tries to Break Starship’s Explosive Losing Streak - https://gizmodo.com/watch-live-as-spacex-tries-to-break-starships-explosive-losing-streak-2000607609
4. After Back-to-Back Flops, SpaceX Says Starship’s 9th Test Flight Is a Go - https://gizmodo.com/after-back-to-back-flops-spacex-s

Prediction After 2025

In [16]:
import requests
import os
from dotenv import load_dotenv
from datetime import datetime

load_dotenv()

def get_next_launch_after_2025():
    """Query SpaceX API to find the next launch after 2025's end."""
    res = requests.get("https://api.spacexdata.com/v4/launches/upcoming")
    upcoming_launches = res.json()

    # Define the target date (end of 2025)
    target_date = datetime(2025, 12, 31, 23, 59, 59)

    for launch in upcoming_launches:
        launch_time_utc = launch.get('date_utc', 'N/A')
        if launch_time_utc != 'N/A':
            try:
                # Attempt to parse the date string into a datetime object
                # Handles both formats 'YYYY-MM-DDTHH:MM:SS.sssZ' and 'YYYY-MM-DDTHH:MM:SZ'
                try:
                    launch_datetime_obj = datetime.strptime(launch_time_utc, "%Y-%m-%dT%H:%M:%S.%fZ")
                except ValueError:
                    launch_datetime_obj = datetime.strptime(launch_time_utc, "%Y-%m-%dT%H:%M:%SZ")

                # Check if the launch date is after the target date
                if launch_datetime_obj > target_date:
                    return {
                        "location_id": launch['launchpad'],
                        "time": launch_time_obj.strftime("%Y-%m-%dT%H:%M:%S.%fZ") # Return the original format or similar
                    }
            except ValueError:
                # Handle cases where date string format is unexpected
                print(f"Could not parse date for launch: {launch_time_utc}")
                continue # Skip to the next launch

    # If no launch found after 2025
    print("No SpaceX launch found after 2025.")
    # Return a default structure to avoid KeyErrors in subsequent functions
    return {
        "location_id": "N/A",
        "time": "N/A"
    }


def get_weather(launch):
    """Using launch's location, find the forecast for launch time."""
    API_KEY = os.getenv("OW_API_KEY")
    # Check if location_id is available before making the API call
    if launch.get('location_id', 'N/A') == 'N/A':
         print("No launch location available for weather forecast.")
         return {
            "latitude": "N/A",
            "longitude": "N/A",
            "forecast": {"list": [{"weather": [{"description": "N/A"}], "main": {"temp": "N/A"}}]}
        }

    launchpad_url = f"https://api.spacexdata.com/v4/launchpads/{launch['location_id']}"
    res = requests.get(launchpad_url)
    launchpad = res.json()

    # Check if latitude and longitude are available
    lat = launchpad.get('latitude', 'N/A')
    lon = launchpad.get('longitude', 'N/A')

    if lat == 'N/A' or lon == 'N/A':
        print(f"Could not get latitude or longitude for launchpad: {launch.get('location_id', 'N/A')}")
        return {
            "latitude": lat,
            "longitude": lon,
            "forecast": {"list": [{"weather": [{"description": "N/A"}], "main": {"temp": "N/A"}}]}
        }


    res = requests.get(
        "https://api.openweathermap.org/data/2.5/forecast",
        params={"appid": API_KEY, "lat": lat, "lon": lon, "units": "metric"}
    )
    forecast = res.json()

    # Add error handling to check if the API call was successful and the expected keys exist
    if res.status_code != 200 or 'list' not in forecast:
        print(f"Error fetching weather data: {forecast}")
        # You might want to raise an exception or return a default value here
        # For now, returning an empty structure to prevent the KeyError in summarize
        return {
            "latitude": lat,
            "longitude": lon,
            "forecast": {"list": [{"weather": [{"description": "N/A"}], "main": {"temp": "N/A"}}]}
        }

    return {
        "latitude": lat,
        "longitude": lon,
        "forecast": forecast
    }

def get_news(launch):
    """Search News API for articles related to SpaceX launch."""
    API_KEY = os.getenv("NEWS_API_KEY")
    res = requests.get("https://newsapi.org/v2/everything",
                        params={"q": "SpaceX launch", "apiKey": API_KEY})
    articles = res.json().get("articles", [])[:5]  # first 5 articles, use .get() for safety
    return articles

def summarize(launch, weather, news):
    """Combine all into a summary."""
    # Use .get() with default values to safely access nested keys
    weather_description = weather.get('forecast', {}).get('list', [{}])[0].get('weather', [{}])[0].get('description', 'N/A').lower() # Get description and convert to lowercase
    temp = weather.get('forecast', {}).get('list', [{}])[0].get('main', {}).get('temp', 'N/A')
    launch_time_utc = launch.get('time', 'N/A')

    # Predict launch viability based on simple weather conditions
    launch_prediction = "Likely proceeding"
    if "rain" in weather_description or "storm" in weather_description or "thunderstorm" in weather_description or "snow" in weather_description or "fog" in weather_description:
        launch_prediction = "Potentially delayed due to weather"
    # Add a check if launch data is not available
    if launch_time_utc == 'N/A':
         launch_prediction = "Cannot predict, launch data not available"


    # Extract and format the launch date
    launch_date = "N/A"
    if launch_time_utc != 'N/A':
        try:
            # Assuming the format is like 'YYYY-MM-DDTHH:MM:SS.sssZ'
            launch_datetime_obj = datetime.strptime(launch_time_utc, "%Y-%m-%dT%H:%M:%S.%fZ")
            launch_date = launch_datetime_obj.strftime("%Y-%m-%d") # Format to YYYY-MM-DD
        except ValueError:
            try:
                # Try parsing without milliseconds if the first format fails
                launch_datetime_obj = datetime.strptime(launch_time_utc, "%Y-%m-%dT%H:%M:%SZ")
                launch_date = launch_datetime_obj.strftime("%Y-%m-%d")
            except ValueError:
                 launch_date = "N/A (Could not parse date)"


    summary = f"""🚀 Next launch:
    - Predicted Launch Date: {launch_date}
    - Location (latitude:{weather.get('latitude', 'N/A')} , longitude:{weather.get('longitude', 'N/A')})
    - Time: {launch_time_utc}
    - Weather forecast at launch: {weather_description.capitalize()} with temperature {temp}°C
    - Launch Weather Prediction: {launch_prediction}

    📰 News related to launch:
    """
    if not news: # Check if news list is empty
        summary += "No news articles found related to the launch.\n"
    else:
        for i, article in enumerate(news, 1):
            # Use .get() with default values for article details as well
            title = article.get('title', 'No Title')
            url = article.get('url', '#')
            summary += f"{i}. {title} - {url}\n"

    return summary

# Run pipeline
launch = get_next_launch_after_2025() # Call the new function
weather = get_weather(launch)
news = get_news(launch)

final_report = summarize(launch, weather, news)
print(final_report)

No SpaceX launch found after 2025.
No launch location available for weather forecast.
🚀 Next launch:
    - Predicted Launch Date: N/A
    - Location (latitude:N/A , longitude:N/A)
    - Time: N/A
    - Weather forecast at launch: N/a with temperature N/A°C
    - Launch Weather Prediction: Cannot predict, launch data not available

    📰 News related to launch:
    1. SpaceX Tests Starship Fixes After Back-to-Back Failures - https://www.wired.com/story/spacex-tests-starship-fixes-after-back-to-back-failures/
2. Florida Environmentalists Sound Alarm on SpaceX Bid to Launch 120 Rockets a Year - https://gizmodo.com/florida-environmentalists-sound-alarm-on-spacex-bid-to-launch-120-rockets-a-year-2000603090
3. Watch Live as SpaceX Tries to Break Starship’s Explosive Losing Streak - https://gizmodo.com/watch-live-as-spacex-tries-to-break-starships-explosive-losing-streak-2000607609
4. After Back-to-Back Flops, SpaceX Says Starship’s 9th Test Flight Is a Go - https://gizmodo.com/after-back-to-